## Advanced Interferogram Processing

Here we will go over some of the more advanced interferometer data processing methods available in prysm.  Many unrelated techniques will be covered here.  A "master interferogram" is going to be the starting point for each process.  This also demonstrates how you can create checkpoints in your own processing routines, if you wish.

As always, we begin with a few imports.


In [ ]:
import numpy as np

from prysm.interferogram import Interferogram
from prysm.sample_data import sample_files
from prysm.geometry import circle

Now we're going to make the master dataset,

In [ ]:
path = sample_files('dat')
master = Interferogram.from_zygo_dat(path)
master.recenter()
master.mask(circle(20, master.r))
master.crop()
master.remove_piston()
master.remove_tiptilt()
master.plot2d()

Two things should be noted here:

- The area outside the clear aperture is filled with NaN values

- There is a region with data dropout within the clear aperture

For reference, the PVr and RMS in units of nanometer are:

In [ ]:
master.pvr(), master.rms

PVr is a method because the user may wish to control the normalization radius used in the Zernike fit that is part of the definition of PVr.  Before continuing, let's look at all the things we can do with our interferogram:

In [ ]:
[s for s in dir(master) if not s.startswith('_')]

Some of these things (`x,y,r,t`) represent the coordinate grid.  Some others (`Sa`, `pv`, `PVr`, `rms`, `strehl`, `std`, `dropout_percentage`, `total_integrated_scatter`) are statistical descriptions of the data.  The low-order removal methods were already discussed.  We have one alternative visualization method:

In [ ]:
master.interferogram(tilt_waves=(1,1))

Some like to view these synthetic interferograms.  The method allows the visibility, number of passes, and any extra tilt fringes to be controlled.

The first thing you may want to do is evaluate the bandlimited RMS value of the data.  We can do so by first filling our NaNs with zero and then using the method.  Here we'll look in the 1 to 10 mm spatial period bandpass.  Equivalent arguments are provided for frequencies, instead of periods.

In [ ]:
scratch = master.copy()
scratch.fill()
scratch.bandlimited_rms(1, 10)

This value is in nanometers, and is roughly half the total RMS of our part.  We can filter the data to the asme spatial period range and see that we get a similar answer:

In [ ]:
# filter only takes frequencies
scratch.filter((1/10, 1), typ='bandpass')
mask = np.isfinite(master.data)
scratch.mask(mask)
scratch.plot2d()
scratch.rms

The value we get by this computation is a bit lower than the value we got with the bandlimited RMS function (about 15% lower).  The reason for this is because spectral methods have finite out-of-band rejection.  While prysm has significantly higher out of band rejection than the software sold with interferometers (> 60 dB higher), it is still finite, especially when the critical frequencies are near the lower or upper sampling limits of the data.  We can view the PSD before and after filtering to see things more clearly:

In [ ]:
scratch2 = master.copy()
scratch2.fill()
psd_no_filter = scratch2.psd()

fig, ax = psd_no_filter.slices().plot('azavg')
scratch.fill()
psd_filter = scratch.psd()
psd_filter.slices().plot('azavg', fig=fig, ax=ax)
ax.set(xlabel='Spatial frequency, cy/mm', ylabel='PSD, nm^2/(cy/mm)^2', yscale='log', xscale='log', ylim=(1e-4,1e5), xlim=(1e-3,10))
ax.legend(['unfiltered', 'filtered'])
ax.grid(True)

In this case, we can see about three orders of magnitude rejection in both out-of-band regions.  This would be considerably larger if the data had more samples (pixels), but the sample file is low resolution:

In [ ]:
print(master.shape)

If we use only low or highpass filters far from the low and high frequency cutoffs, we can achieve stronger rejection:

In [ ]:
scratch = master.copy()
scratch.fill()
scratch.filter(0.1, typ='lp')
fig, ax = psd_no_filter.slices().plot('azavg')
scratch.psd().slices().plot('azavg', fig=fig,ax=ax)

ax.set(yscale='log', xscale='log', ylim=(1e-8,1e5), xlim=(1e-3,10))
ax.legend(['unfiltered', 'filtered'])
ax.grid(True)
ax.axvline(0.1)

The small gain in power in the bandpass is a computational artifact (spectral leakage) and once again related to the low resolution of this interferogram.  We can see a rejection from about 10^2 to 10^-7 by the time we reach 2x the cutoff frequency, or -80dB.

The last processing feature built into the Interferogram class is for spike clipping.  This works the same way it does in MetroPro and Mx:

In [ ]:
scratch = master.copy()
scratch.spike_clip(3)  # 3 sigma is the default, too.

A thoughtful API for polynomial fitting as part of the interferogram interface has not been designed yet.  If you strongly desire one, please do a design and submit a pull request on github.  This _does not_ mean polynomial fitting is not possible.  Here we show fitting some low order Zernike polynomials,

In [ ]:
from prysm.polynomials import (
    fringe_to_nm,
    zernike_nm_sequence,
    lstsq,
    sum_of_2d_modes
)
from prysm.polynomials.zernike import barplot_magnitudes, zernikes_to_magnitude_angle

from prysm.util import rms

In [ ]:
r, t, data = master.r, master.t, master.data
normalization_radius = master.support/2
r = r / normalization_radius
fringe_indices = range(1,37)
nms = [fringe_to_nm(j) for j in fringe_indices]
modes = list(zernike_nm_sequence(nms, r, t))
fit = lstsq(modes, data)

pak = [[*nm, c] for nm, c in zip(nms, fit)]
magnitudes = zernikes_to_magnitude_angle(pak)
barplot_pak = {k: v[0] for k, v in magnitudes.items()}
barplot_magnitudes(barplot_pak)

We can view the projection of various Zernike bandpasses:

In [ ]:
from matplotlib import pyplot as plt
low_order_projection = sum_of_2d_modes(modes[:10], fit[:10])
low_order_projection[~mask] = np.nan
plt.imshow(low_order_projection)

In [ ]:
mid_order_projection = sum_of_2d_modes(modes[10:22], fit[10:22])
mid_order_projection[~mask] = np.nan
plt.imshow(mid_order_projection)

In [ ]:
high_order_projection = sum_of_2d_modes(modes[22:], fit[22:])
high_order_projection[~mask] = np.nan
plt.imshow(high_order_projection)

As well as the total fit Zernike component:

In [ ]:
total_projection = sum_of_2d_modes(modes, fit)
total_projection[~mask] = np.nan
plt.imshow(total_projection)

And the fit error:

In [ ]:
fit_err_map = master.data - total_projection
plt.imshow(fit_err_map, clim=(-50,50), cmap='RdBu')
rms(fit_err_map) # nm

We can do the same with other polynomial bases,

In [ ]:
from prysm.polynomials import Q2d_sequence

In [ ]:
modesQ = list(Q2d_sequence(nms, r, t))
fitQ = lstsq(modesQ, data)

In [ ]:
total_projection = sum_of_2d_modes(modesQ, fitQ)
total_projection[~mask] = np.nan
plt.imshow(total_projection)

In [ ]:
fit_err_map = master.data - total_projection
plt.imshow(fit_err_map, clim=(-50,50), cmap='RdBu')
rms(fit_err_map) # nm

We can see that the common polynomial framework of prysm made it trivial to swap out one polynomial basis for another.

As a final note, the metadata from the dat file is available in a python-friendly format:

In [ ]:
master.meta

As well, the actual intensity camera data is available:

In [ ]:
plt.imshow(master.intensity)

Wrapping up, in this how-to we explored the various advanced processing routines for interferometer data present in prysm.  We did not cover computing a PSF, MTF, or other downstream optical data products from the data.  The `.data` and `.dx` attributes can be used to import the numerical data into the propagation routines of prysm.  The facilities here can be combined to replace the software that comes with an interferometer to perform both basic and advanced processing alike.